# Queries and Selections

## Load movielens datasets

In [1]:
import os
dset = 'movielens-1m'
fdata = os.path.join(dset, 'ratings.dat.gz')
fitem = os.path.join(dset, 'movies.dat')

In [2]:
# Import CSV files via pandas
import pandas as pd
# pass in column names for each CSV
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(fdata, sep=';', names=r_cols, compression='gzip')

m_cols = ['movie_id', 'title', 'genres']
movies = pd.read_csv(fitem, sep=';', names=m_cols,
                     dtype={'title': object, 'genres': object})

In [3]:
movies.ftypes

movie_id     int64:dense
title       object:dense
genres      object:dense
dtype: object

In [4]:
ratings.ftypes

user_id           int64:dense
movie_id          int64:dense
rating            int64:dense
unix_timestamp    int64:dense
dtype: object

In [5]:
import tables

In [6]:
def to_hdf5_norm(ratings, movies):
    
    class Ratings(tables.IsDescription):
        user_id = tables.Int32Col(pos=0)
        movie_id = tables.Int32Col(pos=1)
        rating = tables.Int8Col(pos=2)
        unix_timestamp = tables.Int64Col(pos=3)
    
    class Movies(tables.IsDescription):
        movie_id = tables.Int32Col(pos=0)
        title = tables.StringCol(100, pos=1)
        genres = tables.StringCol(50, pos=2)
    
    filters = tables.Filters(complevel=5, complib="blosc:zstd")
    with tables.open_file("movielens-norm.h5", "w", filters=filters) as f:
        table_ratings = f.create_table(f.root, "ratings", Ratings)
        table_ratings.append([ratings[col].values for col in ratings.ftypes.keys()])
        table_movies = f.create_table(f.root, "movies", Movies)
        table_movies.append([movies[col].values for col in movies.ftypes.keys()])

In [7]:
to_hdf5_norm(ratings, movies)

In [8]:
!ptdump -v -R0,10 movielens-norm.h5

/ (RootGroup) ''
/movies (Table(3883,), shuffle, blosc:zstd(5)) ''
  description := {
  "movie_id": Int32Col(shape=(), dflt=0, pos=0),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (425,)
  Data dump:
[0] (1, b'Toy Story (1995)', b"Animation|Children's|Comedy")
[1] (2, b'Jumanji (1995)', b"Adventure|Children's|Fantasy")
[2] (3, b'Grumpier Old Men (1995)', b'Comedy|Romance')
[3] (4, b'Waiting to Exhale (1995)', b'Comedy|Drama')
[4] (5, b'Father of the Bride Part II (1995)', b'Comedy')
[5] (6, b'Heat (1995)', b'Action|Crime|Thriller')
[6] (7, b'Sabrina (1995)', b'Comedy|Romance')
[7] (8, b'Tom and Huck (1995)', b"Adventure|Children's")
[8] (9, b'Sudden Death (1995)', b'Action')
[9] (10, b'GoldenEye (1995)', b'Action|Adventure|Thriller')
/ratings (Table(1000209,), shuffle, blosc:zstd(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "movie_id": 

In [9]:
# create one merged DataFrame
lens = pd.merge(movies, ratings)

In [10]:
lens.ftypes

movie_id           int64:dense
title             object:dense
genres            object:dense
user_id            int64:dense
rating             int64:dense
unix_timestamp     int64:dense
dtype: object

In [11]:
def to_hdf5_denorm(lens):
    
    class Lens(tables.IsDescription):
        user_id = tables.Int32Col(pos=0)
        rating = tables.Int8Col(pos=1)
        unix_timestamp = tables.Int64Col(pos=2)
        title = tables.StringCol(100, pos=3)
        genres = tables.StringCol(50, pos=4)
    
    filters = tables.Filters(complevel=5, complib="blosc:zstd")
    with tables.open_file("movielens-denorm.h5", "w", filters=filters) as f:
        table_lens = f.create_table(f.root, "lens", Lens)
        table_lens.append([lens[col].values for col in Lens().columns.keys()])


In [12]:
to_hdf5_denorm(lens)

In [13]:
!ptdump -v -R0,10 movielens-denorm.h5

/ (RootGroup) ''
/lens (Table(1000209,), shuffle, blosc:zstd(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "rating": Int8Col(shape=(), dflt=0, pos=1),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=2),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=3),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=4)}
  byteorder := 'little'
  chunkshape := (402,)
  Data dump:
[0] (1, 5, 978824268, b'Toy Story (1995)', b"Animation|Children's|Comedy")
[1] (6, 4, 978237008, b'Toy Story (1995)', b"Animation|Children's|Comedy")
[2] (8, 4, 978233496, b'Toy Story (1995)', b"Animation|Children's|Comedy")
[3] (9, 5, 978225952, b'Toy Story (1995)', b"Animation|Children's|Comedy")
[4] (10, 5, 978226474, b'Toy Story (1995)', b"Animation|Children's|Comedy")
[5] (18, 4, 978154768, b'Toy Story (1995)', b"Animation|Children's|Comedy")
[6] (19, 5, 978555994, b'Toy Story (1995)', b"Animation|Children's|Comedy")
[7] (21, 3, 978139347, b'Toy Story (1995)', b"Anim

In [14]:
!ls -lh movielens*.h5

-rw-r--r--  1 faltet  staff   5.4M May  9 13:44 movielens-denorm.h5
-rw-r--r--  1 faltet  staff   4.3M May  9 13:44 movielens-norm.h5


In [15]:
!du -sh movielens-1m

5.8M	movielens-1m
